## Майнор ВШЭ
## Прикладные задачи анализа данных 2020

### Семинар 15: Обучение с подкреплением: Марковский процесс принятия решений, фреймворк Open AI gym, Q-обучение, аппроксимация Q-функции. 

Обучение с подкреплением (RL - Reinforcement Learning) является направлением машинного обучения и изучает взаимодействие агента, которому необходимо максимизировать долговременный выигрыш в некоторой среде. Агенту не сообщается сведений о правильности действий, как в большинстве задач машинного обучения, вместо этого агент должен определить выгодные действия самостоятельно испробовав их. Испытание действий и отсроченная награда являются основными отличительными признаками RL.
-
<img src="https://github.com/hse-ds/iad-applied-ds/raw/master/2020/seminars/seminar15/rlIntro.png" caption="Взаимодействия агента со средой" style="width: 300px;" />

Основные составляющие модели RL:
* $s_t$ -- состояние среды в момент времени $t$,
* $a_t$ -- действие, совершаемое агентом в момент времени $t$,
* $r_t$ -- вознаграждение, получаемое агентом при совершении действия $a_t$,
* $\pi$ -- стратегия, отвечает за выбор действия в конкретном состоянии.

## 1. Марковский процесс принятия решений
В простейших моделях RL среда представляется в виде марковского процесса принятия решений (MDP), где функция перехода определяется как $P(s' |s,a)$, что означает вероятность оказаться в состоянии $s'$ при совершении действия $a$ в состоянии $s$. Вознаграждение теперь определяется как $r(s,a,s')$.

<img src="https://github.com/hse-ds/iad-applied-ds/raw/master/2020/seminars/seminar15/mdp.png" caption="Марковский процесс принятия решений" style="width: 400px;"/>

## 2. Open AI gym

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import random
from IPython.display import clear_output
%matplotlib inline

In [0]:
#устанавливаем библиотеки для визуализации в colab
!apt-get -qq -y install  libnvtoolsext1 > /dev/null
!ln -snf /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so.8.0 /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so
!apt-get -qq -y install xvfb freeglut3-dev ffmpeg> /dev/null
!pip -q install gym
!pip -q install pyglet
!pip -q install pyopengl
!pip -q install pyvirtualdisplay==0.2.5

import matplotlib.pyplot as plt
import matplotlib.animation
import numpy as np
from IPython.display import HTML

In [0]:
# запускаем virtual display
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1024, 768))
display.start()
import os
os.environ["DISPLAY"] = ":" + str(display.display) + "." + str(display.screen)


def show_animation(frames):
    plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi = 72)
    patch = plt.imshow(frames[0])
    plt.axis('off')
    animate = lambda i: patch.set_data(frames[i])
    ani = matplotlib.animation.FuncAnimation(plt.gcf(), animate, frames=len(frames), interval = 50)
    return ani

На семинаре мы будем пользоваться стандартными средами, реализованными в библиотеке OpenAI Gym (https://gym.openai.com).

In [51]:
import gym

# создаем окружение
env = gym.make("MountainCar-v0")
env.reset()
# рисуем картинку
plt.imshow(env.render('rgb_array'))
env.close()

Основные методы класса Env:

* reset() - инициализация окружения, возвращает первое наблюдение (состояние)
* render() - визуализация текущего состояния среды
* step($a$) - выполнить в среде действие a и получить кортеж: $<s_{t+1}, r_t, done, info>$, где done флаг заверешния, а info - дополнительная информация.

Прежде чем начать взаимодействие с окружением, нужно использовать метод reset():

In [52]:
state0 = env.reset()
print("изначальное состояние среды:", state0)
# выполняем действие 2 
new_state, reward, done, _ = env.step(2)
print("новое состояние:", new_state)
print("вознаграждение", reward)

### Среда MountainCar-v0
* Состояния: Type: Box(2)



Num | Observation  | Min  | Max  
----|--------------|------|----   
0   | position     | -1.2 | 0.6
1   | velocity     | -0.07| 0.07


* Действия: Type: Discrete(3)



Num | Action|
----|-------------|
0   | push left   |
1   | no push     |
2   | push right  |

* Вознаграждения: -1 за каждый шаг, пока не достигнута цель 

* Начальное состояние: Случайная позиция от -0.6 до -0.4 с нулевой скоростью.

### Задание 1
В среде MountainCar-v0 мы хотим, чтобы тележка достигла флага. Давайте решим эту задачу, не используя обучение с подкреплением. Модифицируйте код функции act() ниже для выполнения этого задания. Функция получает на вход состояние среды и должна вернуть действие. 

In [0]:
def act(s):
    actions = {'left': 0, 'stop': 1, 'right': 2}
    
    # пример: можем попробовать ехать только влево
    # action = actions['left'] 
    ####### Здесь ваш код ##########
    raise NotImplementedError
    ################################
    return action

Проверяем решение:

In [54]:
# создаем окружение, с ограничением на число шагов, используя wrapper (обертку) TimeLimit
env = gym.wrappers.TimeLimit(gym.make("MountainCar-v0"), max_episode_steps=250)
# проводим инициализацию и запоминаем начальное состояние
s = env.reset()
done = False

frames = []

frames.append(env.render(mode = 'rgb_array'))

while not done:
    # выполняем действие, получаем s, r, done
    s, r, done, _ = env.step(act(s))
    frames.append(env.render(mode = 'rgb_array'))
    # визуализируем окружение
    env.render()

env.close()
if s[0] > 0.47:
    print("Задание выполнено!")
else:
    print("""Исправьте функцию выбора действия!""")

HTML(show_animation(frames).to_jshtml())

## 3. Q-обучение


Одним из наиболее популярных алгоритм обучения на основе временных различий является Q-обучение.Уравнение Беллмана для значения Q-функции записывается как:

$$Q(s,a)=r(s)+\gamma\sum_s'T(s,a,s')\max_{a'}Q(a',s')$$

Уравнение для итерационного обновления значений Q-функции выглядит следующим образом:$$Q(s,a)\leftarrow Q(s,a)+\alpha(r(s)+\gamma\max_{a'}Q(a',s') - Q(s,a)).$$

Алгоритм Q-обучения:
<img src="https://github.com/hse-ds/iad-applied-ds/raw/master/2020/seminars/seminar15/q.png">
где: s - текущее состояние, a - текущее действие, s', a' - состояние и действие на следующем шаге

Для обучения будем использовать среду Taxi-v3. Подробнее про данное окружение можно посмотреть в документации: https://gym.openai.com/envs/Taxi-v3/.

In [71]:
env = gym.make("Taxi-v3")

env.render()

In [0]:
def show_progress(rewards_batch, log, reward_range=None):
    """
    Удобная функция, которая отображает прогресс обучения.
    """

    if reward_range is None:
        reward_range = [-990, +10]
    mean_reward = np.mean(rewards_batch)
    log.append([mean_reward])

    clear_output(True)
    plt.figure(figsize=[8, 4])
    plt.subplot(1, 2, 1)
    plt.plot(list(zip(*log))[0], label='Mean rewards')
    plt.legend(loc=4)
    plt.grid()

    plt.grid()

    plt.show()

In [0]:
# определяем память, в которой будет храниться Q(s,a)
q_table = np.zeros([env.observation_space.n, env.action_space.n])
show_progress
import random
from IPython.display import clear_output

# гиперпараметры алгоритма
alpha = 0.1
gamma = 0.6
epsilon = 0.1

episodes_number = 10001

log = []
rewards_batch = []

### Задание 2

Напишите код для формулы Q-обновления, используя известные: alpha, reward, gamma, next_max, old_value (q_table[state, action])

In [73]:
for i in range(1, episodes_number):
    state = env.reset()

    episode, reward, episode_reward = 0, 0, 0
    done = False
    
    while not done:
        # выбираем действие, используя eps-greedy исследование среды
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # исследуем среду
        else:
            action = np.argmax(q_table[state]) # используем Q-функциthresholdю

        # выполняем действие в среде 
        next_state, reward, done, info = env.step(action) 
        
        # получаем old_value (Q(s,a)) и next_max (max(Q(s', a')))
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        # код для Q-обновления
        # new_value = 
        ####### Здесь ваш код ##########
        raise NotImplementedError
        ################################
        
        q_table[state, action] = new_value

        state = next_state
        episode += 1
        episode_reward += reward
    rewards_batch.append(episode_reward)
     
    if i % 100 == 0:
        show_progress(rewards_batch, log)
        rewards_batch = []
        print(f"Episode: {i}, Reward: {episode_reward}")

Если все сделано правильно, то график должен выйти на плато около 0. А значение вознаграждение будет в диапазоне [-5, 10], за счет случайного выбора начальной позиции такси и пассажира. Попробуйте изменить гиперпараметры и сравните результаты.

## 4. Аппроксимация Q-функции

В данном пункте мы будем использовать библиотеку tensorflow для обучения нейронной сети, хотя можно использовать и любую другую библиотеку.

In [0]:
import sys, os
if 'google.colab' in sys.modules:
    %tensorflow_version 1.x
import pandas as pd

In [57]:
env = gym.make("CartPole-v0").env
env.reset()
n_actions = env.action_space.n
state_dim = env.observation_space.shape

Так как описание состояния в задаче с маятником представляет собой не "сырые" признаки, а уже предобработанные (координаты, углы), нам не нужна для начала сложная архитектура, начнем с такой:
<img src="https://github.com/hse-ds/iad-applied-ds/raw/master/2020/seminars/seminar15/qapp.png">
Для начала попробуйте использовать только полносвязные слои (L.Dense) и простые активационные функции. Сигмоиды и другие функции не будут работать с ненормализованными входными данными.

In [58]:
import tensorflow as tf
import keras
import keras.layers as L
tf.reset_default_graph()
sess = tf.InteractiveSession()
keras.backend.set_session(sess)

In [0]:
network = keras.models.Sequential()
network.add(L.InputLayer(state_dim))
# определяем граф вычислений!
####### Здесь ваш код ##########
raise NotImplementedError
################################

In [0]:
import random
def get_action(state, epsilon=0):
    """
    сэмплируем (eps greedy) действие  
    """
    q_values = network.predict(state[None])[0]
    
    # нужно выбрать действия eps-жадно, как мы делали в табличном Q-обучении
    # action = 
    ####### Здесь ваш код ##########
    raise NotImplementedError
    ################################
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    return action

In [61]:
assert network.output_shape == (None, n_actions), "Убедитесь, что стратегия переводит s -> [Q(s,a0), ..., Q(s, a_last)]"
assert network.layers[-1].activation == keras.activations.linear, "убедитесь, что вы предсказываете q без нелинейности"

# test epsilon-greedy exploration
s = env.reset()
assert np.shape(get_action(s)) == (), "убедитесь, что возвращается только одно действие (типа integer)"
for eps in [0., 0.1, 0.5, 1.0]:
    state_frequencies = np.bincount([get_action(s, epsilon=eps) for i in range(10000)], minlength=n_actions)
    best_action = state_frequencies.argmax()
    assert abs(state_frequencies[best_action] - 10000 * (1 - eps + eps / n_actions)) < 200
    for other_action in range(n_actions):
        if other_action != best_action:
            assert abs(state_frequencies[other_action] - 10000 * (eps / n_actions)) < 200
    print('e=%.1f tests passed'%eps)

Теперь будем приближать Q-функцию агента, минимизируя TD функцию потерь:
$$ L = { 1 \over N} \sum_i (Q_{\theta}(s,a) - [r(s,a) + \gamma \cdot max_{a'} Q_{-}(s', a')]) ^2,$$
где
* $s, a, r, s'$ состояние, действие, вознаграждение и следующее состояние 
* $\gamma$ дисконтирующий множетель.

Основная тонкость состоит в использовании $Q_{-}(s',a')$. Эта та же самая функция, что и $Q_{\theta}$, которая является выходом нейронной сети, но при обучении сети, мы не пропускаем через эти слои градиенты. Для этого используется функция tf.stop_gradient.

In [0]:
# Создаем placeholders для <s, a, r, s'>, 
# не забываем про индикатор окончания эпизода (is_done = True)
states_ph = keras.backend.placeholder(dtype='float32', shape=(None,) + state_dim)
actions_ph = keras.backend.placeholder(dtype='int32', shape=[None])
rewards_ph = keras.backend.placeholder(dtype='float32', shape=[None])
next_states_ph = keras.backend.placeholder(dtype='float32', shape=(None,) + state_dim)
is_done_ph = keras.backend.placeholder(dtype='bool', shape=[None])

In [0]:
# получаем q для всех действий, в текущем состоянии
predicted_qvalues = network(states_ph)

# получаем q-values для выбранного действия
predicted_qvalues_for_actions = tf.reduce_sum(predicted_qvalues * tf.one_hot(actions_ph, n_actions), axis=1)

In [0]:
gamma = 0.99

# применяем сеть для получения q-value для next_states_ph
# predicted_next_qvalues =
####### Здесь ваш код ##########
raise NotImplementedError
################################

# вычисляем V*(next_states) 
# по предсказанным следующим q-values
# next_state_values =
####### Здесь ваш код ##########
raise NotImplementedError
################################

# Вычисляем target q-values для функции потерь 
# target_qvalues_for_actions = 
####### Здесь ваш код ##########
raise NotImplementedError
################################

# для последнего значения в эпизоде используем упрощенную формулу Q(s,a) = r(s,a) 
target_qvalues_for_actions = tf.where(is_done_ph, rewards_ph, target_qvalues_for_actions)

In [0]:
# mean squared error loss, который будем минимизировать
loss = (predicted_qvalues_for_actions - tf.stop_gradient(target_qvalues_for_actions)) ** 2
loss = tf.reduce_mean(loss)

# применяем AdamOptimizer
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

In [0]:
assert tf.gradients(loss, [predicted_qvalues_for_actions])[0] is not None, "убедитесь, что обновление выполняется только для выбранного действия"
assert tf.gradients(loss, [predicted_next_qvalues])[0] is None, "убедитесь, что вы не распространяете градиент Q_(s',a')"
assert predicted_next_qvalues.shape.ndims == 2, "убедитесь, что вы предсказываете q для всех действий,следующего состояния"
assert next_state_values.shape.ndims == 1, "убедитесь, что вы вычислили V(s') как максимум только по оси действий, а не по всем осям"
assert target_qvalues_for_actions.shape.ndims == 1, "что-то не так с целевыми q-значениями, они должны быть вектором"

In [0]:
sess.run(tf.global_variables_initializer())

In [0]:
def generate_session(env, t_max=1000, epsilon=0, train=False):
    total_reward = 0
    s = env.reset()
    
    for t in range(t_max):
        a = get_action(s, epsilon=epsilon)       
        next_s, r, done, _ = env.step(a)
        
        if train:
            sess.run(train_step,{
                states_ph: [s], actions_ph: [a], rewards_ph: [r], 
                next_states_ph: [next_s], is_done_ph: [done]
            })

        total_reward += r
        s = next_s
        if done:
            break
            
    return total_reward

In [0]:
epsilon = 0.5

In [ ]:
for i in range(1, 1000):
    session_rewards = [generate_session(env, epsilon=epsilon, train=True) for _ in range(100)]
    print("epoch #{}\tmean reward = {:.3f}\tepsilon = {:.3f}".format(i, np.mean(session_rewards), epsilon))
    if i % 10 == 0:
          clear_output(wait=True)
    epsilon *= 0.99
    assert epsilon >= 1e-4, "убедитесь, что epsilon не становится < 0"
    
    if np.mean(session_rewards) > 300:
        print("Принято!")
        break

Интерпретация результатов
* mean reward - среднее вознаграждение за эпизод. В случае корректной реализации, этот показатель будет низким первые 5 эпох и только затем будет возрастать и сойдется на 20-30 эпохе в зависимости от архитектуры сети. 
* Если сеть не достигает нужных результатов к концу цикла, попробуйте увеличить число нейронов в скрытом слое или поменяйте $\epsilon$. 
* epsilon -- обеспечивает стремление агента исследовать среду. Можно искусственно изменять малые значения $\epsilon$ при низких результатах на 0.1 - 0.5. 

Полезные ссылки:
* https://github.com/yandexdataschool/Practical_RL
* https://www.youtube.com/playlist?list=PLbWDNovNB5mqFBgq7i3MY6Ui4zudcvNFJ